# 1. Fermionic creation and annihilation operators

This tutorial shows how to construct matrix representations of fermionic operators and numerically verify their (anti-)commutation relations.

In [1]:
# see the README for installation instructions
import fermi_relations as fr

In [2]:
import numpy as np
from scipy import sparse
import scipy.sparse.linalg as spla

In [3]:
# random number generator
rng = np.random.default_rng(42)

In [4]:
# number of fermionic modes
nmodes = 7

# construct sparse matrix representations of fermionic creation, annihilation and number operators
# (implementation is based on the Jordan-Wigner transformation):
clist, alist, nlist = fr.construct_fermionic_operators(nmodes)

`clist`, `alist` and `nlist` are lists of fermionic creation, annihilation and number operators, respectively, represented as sparse matrices (with $L$ the number of fermionic modes):
\begin{align*}
\text{clist} &= \{ a_0^{\dagger}, a_1^{\dagger}, \dots, a_{L-1}^{\dagger} \},\\
\text{alist} &= \{ a_0, a_1, \dots, a_{L-1} \},\\
\text{nlist} &= \{ \hat{n}_0, \hat{n}_1, \dots, \hat{n}_{L-1} \}.
\end{align*}

In [5]:
clist

[<Compressed Sparse Row sparse matrix of dtype 'float64'
 	with 64 stored elements and shape (128, 128)>,
 <Compressed Sparse Row sparse matrix of dtype 'float64'
 	with 64 stored elements and shape (128, 128)>,
 <Compressed Sparse Row sparse matrix of dtype 'float64'
 	with 64 stored elements and shape (128, 128)>,
 <Compressed Sparse Row sparse matrix of dtype 'float64'
 	with 64 stored elements and shape (128, 128)>,
 <Compressed Sparse Row sparse matrix of dtype 'float64'
 	with 64 stored elements and shape (128, 128)>,
 <Compressed Sparse Row sparse matrix of dtype 'float64'
 	with 64 stored elements and shape (128, 128)>,
 <Compressed Sparse Row sparse matrix of dtype 'float64'
 	with 64 stored elements and shape (128, 128)>]

In [6]:
# verify that the matrices in 'clist' are indeed the respective adjoints of the matrices in 'alist':
[spla.norm(clist[i] - alist[i].conj().T) for i in range(nmodes)]

[np.float64(0.0),
 np.float64(0.0),
 np.float64(0.0),
 np.float64(0.0),
 np.float64(0.0),
 np.float64(0.0),
 np.float64(0.0)]

In [7]:
# verify correct definition of the number operators: n_i = a_i† a_i
[spla.norm(nlist[i] - clist[i] @ alist[i]) for i in range(nmodes)]

[np.float64(0.0),
 np.float64(0.0),
 np.float64(0.0),
 np.float64(0.0),
 np.float64(0.0),
 np.float64(0.0),
 np.float64(0.0)]

In [8]:
# eigenvalues of a number operator are 0 and 1:
np.linalg.eigvalsh(nlist[2].todense())

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [9]:
# any two creation or annihilation operators anti-commute:
print(sum([spla.norm(fr.anti_comm(ci, cj)) for ci in clist for cj in clist]))
print(sum([spla.norm(fr.anti_comm(ai, aj)) for ai in alist for aj in alist]))

0.0
0.0


In [10]:
# verify the anti-commutation relation of a fermionic creation and annihilation operator: {a_i†, a_j} = delta_{i,j}
spid = sparse.identity(2**nmodes)
print(sum([spla.norm(
    fr.anti_comm(clist[i], alist[j]) - (1 if i == j else 0) * spid)
        for i in range(nmodes)
        for j in range(nmodes)]))

0.0


In [11]:
# verify the commutation relation [n_i, a_i†] = a_i†:
print(sum([spla.norm(
    fr.comm(nlist[i], clist[i]) - clist[i])
        for i in range(nmodes)]))

0.0


A creation operator $a_{\varphi}^{\dagger}$ of an "orbital" $\varphi$ is defined as the linear combination of standard basis creation operators with coefficients given by $\varphi$:
\begin{equation*}
a_{\varphi}^{\dagger} = \sum_{j=0}^{L-1} \varphi_j a_j^{\dagger}, \quad \varphi \in \mathbb{C}^L
\end{equation*}

In [12]:
# define a random "orbital" state
phi = fr.crandn(nmodes, rng)
phi /= np.linalg.norm(phi)

In [13]:
# corresponding creation operator
c_phi = fr.orbital_create_op(phi)
c_phi

<Compressed Sparse Row sparse matrix of dtype 'complex128'
	with 448 stored elements and shape (128, 128)>

In [14]:
# compare with above definition
spla.norm(c_phi - sum(phi[i] * clist[i] for i in range(nmodes)))

np.float64(0.0)

In [15]:
# show some matrix entries
c_phi.todense()

matrix([[ 0.        +0.j        ,  0.        +0.j        ,
          0.        +0.j        , ...,  0.        +0.j        ,
          0.        +0.j        ,  0.        +0.j        ],
        [ 0.0374995 +0.33065429j,  0.        +0.j        ,
          0.        +0.j        , ...,  0.        +0.j        ,
          0.        +0.j        ,  0.        +0.j        ],
        [-0.38196904+0.01936882j,  0.        +0.j        ,
          0.        +0.j        , ...,  0.        +0.j        ,
          0.        +0.j        ,  0.        +0.j        ],
        ...,
        [ 0.        +0.j        ,  0.        +0.j        ,
          0.        +0.j        , ...,  0.        +0.j        ,
          0.        +0.j        ,  0.        +0.j        ],
        [ 0.        +0.j        ,  0.        +0.j        ,
          0.        +0.j        , ...,  0.        +0.j        ,
          0.        +0.j        ,  0.        +0.j        ],
        [ 0.        +0.j        ,  0.        +0.j        ,
          0. 

In [16]:
# number operator of 'phi'
n_phi = fr.orbital_number_op(phi)
n_phi

<Compressed Sparse Row sparse matrix of dtype 'complex128'
	with 1471 stored elements and shape (128, 128)>

In [17]:
# eigenvalues of an "orbital" number operator are 0 and 1, just as for a standard basis number operator:
np.linalg.norm(np.linalg.eigvalsh(n_phi.todense()) - np.array(2**(nmodes - 1) * [0] + 2**(nmodes - 1) * [1]))

np.float64(7.86382474884347e-15)